In [12]:
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from datasets.SP100Stocks import SP100Stocks
from notebooks.models import TGCN, A3TGCN, DCGNN, train, measure_accuracy, get_confusion_matrix

# Buy or Sell? Stocks classifier
The goal of this task is to classify stocks into buy or sell categories based on past performance. The model should predict if the next day is higher or lower than the current day, with past 5 weeks of knowledge. There are no patterns in stock movements, so neural networks have a hard time forecasting the next timestamps. The intuition is that they can, however, capture up/down trends.

## Loading the data
The data from the custom PyG dataset for forecasting is loaded into a PyTorch dataloader.
A "transform" is applied to change the targets `y` of the dataset to a binary buy/sell class instead of the close price. 

In [33]:
def future_close_price_to_buy_sell_class(sample: Data):
	"""
	Transforms the target y to a binary buy (1) if the stock return two weeks ahead was higher that the average market return, else sell (0)
	:param sample: Data sample
	:return: The transformed sample
	"""
	market_return = ((sample.close_price_y[:, -1] - sample.close_price[:, -1]) / sample.close_price[:, -1]).mean()
	sample.returns = ((sample.close_price_y[:, -1] - sample.close_price[:, -1]) / sample.close_price[:, -1]).unsqueeze(1)
	sample.market_return = market_return
	## Multi-class trend classification (down, constant, up)
	# constant_trend_threshold = .01
	# is_down_trend = (sample.returns < -constant_trend_threshold)
	# is_constant_trend = ((sample.returns >= -constant_trend_threshold) * (sample.returns <= constant_trend_threshold))
	# is_up_trend = (sample.returns > constant_trend_threshold)
	# sample.y = (is_constant_trend + is_up_trend * 2).long().squeeze(1)
	# return sample
	# Binary trend classification (down, up)
	sample.y = (sample.returns >= 0).float()
	return sample

In [34]:
weeks_ahead = 2

dataset = SP100Stocks(future_window=weeks_ahead * 5, force_reload=True, transform=future_close_price_to_buy_sell_class)
dataset, dataset[0]

(SP100Stocks(1182),
 Data(x=[100, 8, 25], edge_index=[2, 524], y=[100, 1], edge_weight=[524], close_price=[100, 25], close_price_y=[100, 10], returns=[100, 1], market_return=0.019632836803793907))

In [35]:
for i in range(0, 10):
	# print(f"Stock return: {dataset[i].returns[i].item() * 100:.2f}%, trend: {['Down', 'Constant', 'Up'][dataset[i].y[i].item()]}")
	print(f"Stock return: {dataset[i].returns[i].item() * 100:.2f}%, trend: {['Down', 'Up'][int(dataset[i].y[i].item())]}")

Stock return: 1.08%, trend: Up
Stock return: -1.74%, trend: Down
Stock return: -0.38%, trend: Down
Stock return: 1.92%, trend: Up
Stock return: -0.96%, trend: Down
Stock return: 5.56%, trend: Up
Stock return: -0.17%, trend: Down
Stock return: -5.16%, trend: Down
Stock return: -4.91%, trend: Down
Stock return: 2.96%, trend: Up


In [44]:
train_part = .9
batch_size = 32

train_dataset, test_dataset = dataset[:int(train_part * len(dataset))], dataset[int(train_part * len(dataset)):]
print(f"Train dataset: {len(train_dataset)}, Test dataset: {len(test_dataset)}")
train_dataloader, test_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True), DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)

Train dataset: 1063, Test dataset: 119


## Training
The previously implemented models are used, trained using the training dataset and the Adam optimizer. The `weight_decay` parameter is used for L2 regularization, to follow the T-GCN papers methodology. The loss is calculated using the Binary Cross Entropy (BCE) loss function.

In [45]:
in_channels, out_channels, hidden_size, layers_nb, dropout = dataset[0].x.shape[-2], 1, 16, 2, .3
model = TGCN(in_channels, out_channels, hidden_size, layers_nb)

lr, weight_decay, num_epochs = 0.005, 1e-5, 64

# criterion = nn.CrossEntropyLoss()  # for multi-class classification
criterion = nn.BCEWithLogitsLoss()  # for binary up/down classification
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
model

TGCN(
  (cells): ModuleList(
    (0): TGCNCell(
      (gcn): GAT(
        (convs): ModuleList(
          (0): GATv2Conv(8, 16, heads=1)
          (1): GATv2Conv(16, 16, heads=1)
        )
      )
      (lin_u): Linear(in_features=40, out_features=16, bias=True)
      (lin_r): Linear(in_features=40, out_features=16, bias=True)
      (lin_c): Linear(in_features=40, out_features=16, bias=True)
    )
    (1): TGCNCell(
      (gcn): GAT(
        (convs): ModuleList(
          (0-1): 2 x GATv2Conv(16, 16, heads=1)
        )
      )
      (lin_u): Linear(in_features=48, out_features=16, bias=True)
      (lin_r): Linear(in_features=48, out_features=16, bias=True)
      (lin_c): Linear(in_features=48, out_features=16, bias=True)
    )
  )
  (out): Sequential(
    (0): Linear(in_features=16, out_features=1, bias=True)
    (1): Identity()
  )
)

In [46]:
train(model, optimizer, criterion, train_dataloader, test_dataloader, num_epochs, "BuyOrSell", measure_acc=True)

Epochs: 100%|██████████| 64/64 [2:09:03<00:00, 120.99s/it, Batch=100.0%]  


## Results

### Results on train data

In [47]:
full_train_data = next(iter(DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=True)))
acc, cm = measure_accuracy(model, full_train_data), get_confusion_matrix(model, full_train_data)

print(f"Train accuracy: {acc * 100:.1f}%\nTrain confusion matrix:\n{cm}")

Train accuracy: 77.7%
Train confusion matrix:
[[33011 14409]
 [ 9268 49612]]


### Results on test data

In [48]:
acc, cm = measure_accuracy(model, next(iter(train_dataloader))), get_confusion_matrix(model, next(iter(train_dataloader)))

print(f"Test accuracy: {acc * 100:.1f}%\nTest confusion matrix:\n{cm}")

Test accuracy: 77.0%
Test confusion matrix:
[[ 912  418]
 [ 249 1621]]
